In [48]:
import os

import pygraphviz as pgv
from tokenizers import Tokenizer

tokenizer_uni = Tokenizer.from_file('../data/processed/tokenizer/words_unigram_5000.tokenizer.json')
with open('../data/processed/word_freqs/freq_1000000_oshhamaho.txt') as f:
    words = f.read().split('\n')

In [49]:
FSM_FILE_DIR = "../data/processed/word_fsm/"
os.makedirs(FSM_FILE_DIR, exist_ok=True)

In [50]:
def customize_node(token, token_index, root_index, custom_tokens=None):
    """
    Кастомизация узла по признакам токена
    :param token: 
    :param token_index: 
    :param root_index: 
    :param custom_tokens: 
    :return: 
    """
    custom_tokens = custom_tokens or []

    node_kw = {
        'color': 'black',
        'shape': 'circle'
    }
    if token_index == root_index:
        node = token
        node_kw['color'] = 'dodgerblue'
    elif token_index < root_index:
        node = f'{token} ^('  # отмечаем что токен находится левее корня
        node_kw['color'] = 'webgreen'
    else:
        node = f')^ {token}'  # отмечаем что токен находится правее корня
        node_kw['color'] = 'firebrick'

    if token in custom_tokens:  # помечаем интересующие нас токены прямоугольником
        node_kw['shape'] = "rect"
    
    return node, node_kw


def add_word_tokens_to_graph(graph, root, tokens):
    """
    Добавление токенов слова в граф
    :param graph: 
    :param root: 
    :param tokens: 
    :return: 
    """
    state = "^"
    root_index = tokens.index(root)
    for index, token in enumerate(tokens):
        node, node_kw = customize_node(token, token_index=index, root_index=root_index)
        graph.add_node(node, **node_kw)
        graph.add_edge(state, node, label=token, color=node_kw.get('color'))

        state = node

    graph.get_node(state).attr["shape"] = "doublecircle"


def build_words_fsm_by_tokens(words, root, tokenizer):
    """
    Создание автомата по токенам слов
    :param words: 
    :param root: 
    :param tokenizer: 
    :return: 
    """
    graph = pgv.AGraph(directed=True, rankdir="LR")
    graph.add_node("start", shape="circle")

    for word in words:
        tokens = tokenizer.encode(word).tokens
        if root not in tokens:
            continue

        add_word_tokens_to_graph(graph=graph, root=root, tokens=tokens)

    return graph


def save_fsm(fsm, root, filedir=FSM_FILE_DIR):
    filename = root
    filepath = os.path.join(filedir, filename)

    fsm.layout(prog="dot")
    # fsm.draw(f"{filepath}.png")  # для больших графов png получаются гигантскими
    fsm.draw(f"{filepath}.svg", 'svg:cairo')

In [51]:
word_root = 'гъэджэгу'
root_words = [word for word in words if word_root in word]

tokenizer_uni.add_tokens([word_root])

fsm = build_words_fsm_by_tokens(words=root_words, root=word_root, tokenizer=tokenizer_uni)
save_fsm(fsm=fsm, root=word_root)

Визуализация морфем в виде графа улучшит понимание структуры слов и морфологических связей